In [7]:
# Import necessary libraries
import numpy as np
from matplotlib import pyplot as plt
import time
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras import models, layers
#from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# for data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#from tensorflow.keras.models import load_modelfrom 
#from tensorflow.keras.utils import to_categoricaltensorflow.keras.callbacks import EarlyStopping
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#from sklearn.decomposition import PCA

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load and preprocess CIFAR-10 data
(train_images, train_labels), (testing_images, testing_labels) = cifar10.load_data()

# 40k training
# 10k validating
# 10k testing
# Further split training set into training and validation sets
train_images, validation_images, train_labels, validation_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)
# test_size=0.2 is the proportion of the dataset to be used for validation set
# random_state=42 the split will always produce the same subsets of training and validation data
# ensuring consistency in your experiment

In [ ]:
# Normalize image data to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
validation_images = validation_images.astype('float32') / 255.0
testing_images = testing_images.astype('float32')/255.0

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=4,          # Stop training if val_loss doesn't improve for 3 consecutive epochs
    restore_best_weights=True  # Restore weights from the epoch with the best val_loss
)

model_checkpoint = ModelCheckpoint(
    filepath='best_model2.keras',  # Save the best model to a file
    monitor='val_loss',       # Save the model with the lowest validation loss
    save_best_only=True       # Only save when there's an improvement
)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    #brightness_range=[0.9, 1.1],
    #shear_range=0.1, # distorting the image on an axes
    #zoom_range=0.1,
    #fill_mode='nearest', # filling the extra pixels using nearest neighbor
    #channel_shift_range=5.0 # colos playing
)

datagen.fit(train_images)

In [ ]:
# Model Definition
input_shape = (32, 32, 3)
model2 = models.Sequential()

# convolutional layers with Batch Normalization
model2.add(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D((2, 2)))

# a dropout layer for regularization
model2.add(Dropout(0.5))

# Flatten and fully connected layers
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))

# Output layer for 10 classes
model2.add(Dense(10, activation='softmax'))

# Compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model2.summary()

In [ ]:
# TRAINING DATA SET 1st RUN

start_time = time.time()

history = model2.fit(
    train_images,
    train_labels,
    epochs=40, # EARLY_STOPPING WILL DEAL WITH OVERFITTING
    validation_data=(validation_images, validation_labels),
    batch_size=64,
    shuffle=True,
    callbacks=[early_stopping, model_checkpoint]
)
# Print the epoch where training stopped
print(f"Training stopped at epoch {len(history.history['loss'])}")

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time: {total_time:.2f} seconds")

# Extract accuracy and loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

In [ ]:
# TESTING
test_loss, test_accuracy = model2.evaluate(testing_images, testing_labels)

In [ ]:
# Plot training and validation accuracy/loss
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.grid()
plt.plot(epochs, train_accuracy, color='orange', label='Training Accuracy', marker='o')
plt.plot(epochs, val_accuracy, color='purple', label='Validation Accuracy', marker='o')
plt.axhline(y=test_accuracy, color='r', linestyle='--', label='Test Accuracy')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.grid()
plt.plot(epochs, train_loss, color='orange', label='Training Loss', marker='o')
plt.plot(epochs, val_loss, color='purple', label='Validation Loss', marker='o')
plt.axhline(y=test_loss, color='r', linestyle='--', label='Test Loss')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# redefine earlystopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=4,          # Stop training if val_loss doesn't improve for 3 consecutive epochs
    restore_best_weights=True  # Restore weights from the epoch with the best val_loss
)

# TRAINING DATA SET 2nd RUN

start_time = time.time()

history = model2.fit(
    train_images,
    train_labels,
    epochs=20, # EARLY_STOPPING WILL DEAL WITH OVERFITTING
    validation_data=(validation_images, validation_labels),
    batch_size=64,
    shuffle=True,
    callbacks=[early_stopping, model_checkpoint]
)
# Print the epoch where training stopped
print(f"Training stopped at epoch {len(history.history['loss'])}")

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time: {total_time:.2f} seconds")

# Extract accuracy and loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

In [ ]:
# TESTING
test_loss, test_accuracy = model2.evaluate(testing_images, testing_labels)

In [ ]:
# Plot training and validation accuracy/loss
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.grid()
plt.plot(epochs, train_accuracy, color='orange', label='Training Accuracy', marker='o')
plt.plot(epochs, val_accuracy, color='purple', label='Validation Accuracy', marker='o')
plt.axhline(y=test_accuracy, color='r', linestyle='--', label='Test Accuracy')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.grid()
plt.plot(epochs, train_loss, color='orange', label='Training Loss', marker='o')
plt.plot(epochs, val_loss, color='purple', label='Validation Loss', marker='o')
plt.axhline(y=test_loss, color='r', linestyle='--', label='Test Loss')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# redefine earlystopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=4,          # Stop training if val_loss doesn't improve for 3 consecutive epochs
    restore_best_weights=True  # Restore weights from the epoch with the best val_loss
)

# AUGMENTED TRAINING DATA SET 1st RUN

start_time = time.time()

# using the augmented images-data set
history = model2.fit(
    datagen.flow(train_images, train_labels, batch_size=64),
    epochs=50, # EARLY_STOPPING WILL DEAL WITH OVERFITTING
    validation_data=(validation_images, validation_labels),
    callbacks=[early_stopping, model_checkpoint]
)
# Print the epoch where training stopped
print(f"Training stopped at epoch {len(history.history['loss'])}")

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time: {total_time:.2f} seconds")

# Extract accuracy and loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

In [ ]:
# TESTING
test_loss, test_accuracy = model2.evaluate(testing_images, testing_labels)

In [ ]:
# Plot training and validation accuracy/loss
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.grid()
plt.plot(epochs, train_accuracy, color='orange', label='Training Accuracy', marker='o')
plt.plot(epochs, val_accuracy, color='purple', label='Validation Accuracy', marker='o')
plt.axhline(y=test_accuracy, color='r', linestyle='--', label='Test Accuracy')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.grid()
plt.plot(epochs, train_loss, color='orange', label='Training Loss', marker='o')
plt.plot(epochs, val_loss, color='purple', label='Validation Loss', marker='o')
plt.axhline(y=test_loss, color='r', linestyle='--', label='Test Loss')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()